In [1]:
!pip install -qU crewai crewai_tools langchain-google-genai python-dotenv serpapi

In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = ""
from crewai import LLM, Agent, Task, Crew, Process
from crewai_tools import SerpApiGoogleSearchTool
os.environ["GOOGLE_API_KEY"] = ""
my_llm = LLM(model="gemini/gemini-2.5-flash", temperature=0.2)
print("✅ CrewAI Installed.")

✅ CrewAI Installed.


In [3]:
# ----------------------------
# AGENTS (workers)
# ----------------------------

policy_agent = Agent(
    role="Policy Support Specialist",
    goal="Answer policy/refund/returns/shipping/account questions correctly and clearly.",
    backstory=(
        "You are a support specialist with strong policy knowledge. "
        "You respond concisely, accurately, and ask for missing details when required."
    ),
    llm=my_llm,
    verbose=True
)

empathy_agent = Agent(
    role="Empathy & VIP Support Specialist",
    goal="De-escalate angry customers and provide empathetic, professional responses.",
    backstory=(
        "You handle escalations and VIP customers. You acknowledge feelings, apologize appropriately, "
        "and offer next steps while keeping tone calm and respectful."
    ),
    llm=my_llm,
    verbose=True
)


# ----------------------------
# TASKS (the manager chooses)
# ----------------------------

policy_task = Task(
    description=(
        "You will receive a customer email:\n\n{email}\n\n"
        "If this is a policy question (refund/return/shipping/account/how-to), draft a helpful reply.\n"
        "Return STRICT JSON with keys:\n"
        "category (POLICY), priority (LOW/MEDIUM/HIGH), summary, reply, next_questions (array).\n"
        "Do not mention internal routing."
    ),
    expected_output="STRICT JSON as specified.",
    agent=policy_agent
)

empathy_task = Task(
    description=(
        "You will receive a customer email:\n\n{email}\n\n"
        "If this is angry/escalated/VIP/emotional, draft an empathetic reply that de-escalates.\n"
        "Return STRICT JSON with keys:\n"
        "category (EMPATHY), priority (LOW/MEDIUM/HIGH), summary, reply, next_steps (array).\n"
        "Do not mention internal routing."
    ),
    expected_output="STRICT JSON as specified.",
    agent=empathy_agent
)


In [6]:
# ----------------------------
# CREW (Hierarchical = routing)
# ----------------------------

triage_crew = Crew(
    agents=[policy_agent, empathy_agent],
    tasks=[policy_task, empathy_task],
    process=Process.hierarchical,   
    manager_llm=my_llm,                
    verbose=True
)

# ----------------------------
# RUN
# ----------------------------

test_email_1 = """
Hi team, I ordered shoes yesterday but want to cancel and get a refund.
What is your refund policy and how long does it take?
"""

test_email_2 = """
This is ridiculous. Your delivery is late AGAIN. I’m a premium customer and this is unacceptable.
Fix this today or I’m escalating publicly.
"""

print("\n--- RESULT 1 ---")
print(triage_crew.kickoff(inputs={"email": test_email_1}))

print("\n--- RESULT 2 ---")
print(triage_crew.kickoff(inputs={"email": test_email_2}))


--- RESULT 1 ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0b116634-42d5-43b0-9342-9cbac4b3628b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: You will receive a customer email:                                                                       │
│                                                                                                                 │
│                                                                                                                 │
│  Hi team, I ordered shoes yesterday but want to cancel and get a refund.                                        │
│  What is your refund policy and how long does it take?                                                          │
│                                                                                                                 │
│                                                                                                                 │
│  If this is a policy question (refund/return/shipping/account/how-to), draft a helpful reply.                   │
│  Return STRICT JSON with keys:                                                                                  │
│  category (POLICY), priority (LOW/MEDIUM/HIGH), summary, reply, next_questions (array).                         │
│  Do not mention internal routing.                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/devenderswami/lc-env/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Policy Support Specialist                                                                               │
│                                                                                                                 │
│  Task: Draft a helpful reply to a customer asking about our refund policy and how long it takes to process a    │
│  refund. Also, provide the refund policy details, including the duration for processing. The reply should be    │
│  customer-friendly and address all aspects of their query. Additionally, categorize the email as POLICY,        │
│  assign a priority (LOW/MEDIUM/HIGH), summarize the customer's request, and identify any potential next         │
│  questions the customer might have.                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


[2026-01-03 12:16:25][WARNING]: Manager agent should not have tools


╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 0b116634-42d5-43b0-9342-9cbac4b3628b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Exception: Manager agent should not have tools